# 01 - Compute Long Term Averages
For each case, compute the global 3D (`nlat`, `nlon`, `z_t`) mean

## Imports
We include the lines at the beginning to make sure that any updates we make to the `analysis_config.yml` file are reflected in real time for this notebook

In [2]:
%load_ext autoreload
%autoreload 2

import intake
import ast
import yaml
from distributed import Client
from ncar_jobqueue import NCARCluster
import xarray as xr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Spin up a Dask Cluster

In [24]:
cluster = NCARCluster()
cluster.scale(20)
client = Client(cluster)

ERROR! Session/line number was not unique in database. History logging moved to new session 1533


/glade/work/mgrover/miniconda3/envs/cesm-collections-dev/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38696 instead
  warnings.warn(


In [25]:
client

Connection method: Cluster object,Cluster type: PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/38696/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/38696/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.42:45165,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/38696/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Open the Intake-ESM Catalog
In the first notebook, we created a an `intake-esm` catalog which provides a means of accessing our data.

In [5]:
col = intake.open_esm_datastore(
    "/glade/work/mgrover/cesm-validation-catalog.json",
    csv_kwargs={"converters": {"variables": ast.literal_eval}},
    sep="/",
)
col

,unique
component,1
stream,4
date,2501
case,3
member_id,2
frequency,4
variables,545
path,11103


In [10]:
cat = col.search(
    frequency='month_1',
)
cat

,unique
component,1
stream,1
date,1200
case,3
member_id,2
frequency,1
variables,434
path,3600


### Subset the last 20 years of data

In [11]:
dates = sorted(cat.df.date.unique())

In [12]:
sub = cat.search(date=dates[-240:])

### Read in our dataset using `to_dataset_dict()`

In [13]:
dsets = sub.to_dataset_dict(cdf_kwargs={'use_cftime': True, 'chunks': {'time': 60}})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component/stream/case'


In [14]:
dsets['ocn/pop.h/b1850.f19_g17.validation_mct.002']

<xarray.Dataset>
Dimensions:                          (time: 240, d2: 2, moc_comp: 3, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_top: 60, z_w_bot: 60, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320)
Coordinates: (12/14)
    transport_components             (transport_comp) |S384 dask.array<chunksize=(5,), meta=np.ndarray>
    transport_regions                (transport_reg) |S384 dask.array<chunksize=(2,), meta=np.ndarray>
  * time                             (time) object 0094-11-01 00:00:00 ... 00...
  * z_t                              (z_t) float32 500.0 1.5e+03 ... 5.375e+05
  * z_t_150m                         (z_t_150m) float32 500.0 ... 1.45e+04
  * z_w                              (z_w) float32 0.0 1e+03 ... 5e+05 5.25e+05
    ...                               ...
  * lat_aux_grid                     (lat_aux_grid) float32 -79.49 ... 90.0
  * moc_z                            (moc_z) float32 0.0 1e+03 ... 5.5e+05
    ULONG                            (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                             (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLONG                            (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLAT                             (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
Dimensions without coordinates: d2, moc_comp, transport_comp, transport_reg, nlat, nlon
Data variables: (12/486)
    time_bound                       (time, d2) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    moc_components                   (moc_comp) |S384 dask.array<chunksize=(3,), meta=np.ndarray>
    dz                               (z_t) float32 dask.array<chunksize=(60,), meta=np.ndarray>
    dzw                              (z_w) float32 dask.array<chunksize=(60,), meta=np.ndarray>
    KMT                              (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    KMU                              (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ...                               ...
    XBLT                             (time, nlat, nlon) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
    TBLT                             (time, nlat, nlon) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
    BSF                              (time, nlat, nlon) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
    MOC                              (time, transport_reg, moc_comp, moc_z, lat_aux_grid) float32 dask.array<chunksize=(1, 2, 3, 61, 395), meta=np.ndarray>
    N_HEAT                           (time, transport_reg, transport_comp, lat_aux_grid) float32 dask.array<chunksize=(1, 2, 5, 395), meta=np.ndarray>
    N_SALT                           (time, transport_reg, transport_comp, lat_aux_grid) float32 dask.array<chunksize=(1, 2, 5, 395), meta=np.ndarray>
Attributes:
    cell_methods:            cell_methods = time: mean ==> the variable value...
    source:                  CCSM POP2, the CCSM Ocean Component
    model_doi_url:           https://doi.org/10.5065/D67H1H0V
    intake_esm_varname:      ['UVEL', 'UVEL2', 'VVEL', 'VVEL2', 'TEMP', 'dTEM...
    calendar:                All years have exactly  365 days.
    time_period_freq:        month_1
    revision:                $Id$
    contents:                Diagnostic and Prognostic Variables
    title:                   b1850.f19_g17.validation_mct.002
    Conventions:             CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf...
    history:                 none
    intake_esm_dataset_key:  ocn/pop.h/b1850.f19_g17.validation_mct.002

## Loop through the data and compute!
We are computing the average over time, and merging into a single dataset, subsetting for the variables specified in the `analysis_config.yml` file

In [15]:
with open("analysis_config.yml", mode="r") as fptr:
    analysis_config = yaml.safe_load(fptr)
variables = analysis_config['variables']
variables

['TEMP',
 'SALT',
 'HMXL',
 'BSF',
 'IFRAC',
 'SHF',
 'SFWF',
 'SHF_QSW',
 'FG_CO2',
 'PO4',
 'photoC_TOT_zint',
 'photoC_sp_zint',
 'photoC_diat_zint',
 'photoC_diaz_zint',
 'POC_FLUX_100m',
 'diaz_Nfix',
 'SiO2_PROD',
 'CaCO3_FLUX_100m',
 'SiO2_FLUX_100m',
 'ALK',
 'DIC',
 'O2',
 'DOC',
 'DOCr',
 'NH4',
 'NO3',
 'SiO3']

In [16]:
xr.set_options(keep_attrs=True)

In [17]:
ds_list = []
for key in dsets.keys():
    ds = dsets[key]
    mean = ds.mean(dim='time')
    out = mean[variables]
    out.attrs['intake_esm_varname'] = variables
    ds_list.append(out)

In [18]:
merged_ds = xr.concat(ds_list, dim='case')

We also want to make sure that we keep the title, or case information

In [20]:
cases = []
for ds in ds_list:
    cases.append(ds.title)

In [74]:
merged_ds['case'] = cases

In [23]:
merged_ds['case'] = cases

In [26]:
merged_ds.persist()

<xarray.Dataset>
Dimensions:           (case: 3, z_t: 60, nlat: 384, nlon: 320, z_t_150m: 15)
Coordinates:
  * z_t               (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m          (z_t_150m) float32 500.0 1.5e+03 ... 1.35e+04 1.45e+04
    ULONG             (nlat, nlon) float64 321.1 322.3 323.4 ... 319.6 320.0
    ULAT              (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.41 72.41
    TLONG             (nlat, nlon) float64 320.6 321.7 322.8 ... 319.4 319.8
    TLAT              (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.19 72.19
  * case              (case) <U34 'b1850.f19_g17.validation_mct.004' ... 'b18...
Dimensions without coordinates: nlat, nlon
Data variables: (12/27)
    TEMP              (case, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
    SALT              (case, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
    HMXL              (case, nlat, nlon) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
    BSF               (case, nlat, nlon) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
    IFRAC             (case, nlat, nlon) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
    SHF               (case, nlat, nlon) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
    ...                ...
    O2                (case, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
    DOC               (case, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
    DOCr              (case, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
    NH4               (case, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
    NO3               (case, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
    SiO3              (case, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
Attributes:
    cell_methods:            cell_methods = time: mean ==> the variable value...
    source:                  CCSM POP2, the CCSM Ocean Component
    model_doi_url:           https://doi.org/10.5065/D67H1H0V
    intake_esm_varname:      ['TEMP', 'SALT', 'HMXL', 'BSF', 'IFRAC', 'SHF', ...
    calendar:                All years have exactly  365 days.
    time_period_freq:        month_1
    revision:                $Id$
    contents:                Diagnostic and Prognostic Variables
    title:                   b1850.f19_g17.validation_mct.004
    Conventions:             CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf...
    history:                 none
    intake_esm_dataset_key:  ocn/pop.h/b1850.f19_g17.validation_mct.004

### Export our data
We output our dataset to zarr!

In [27]:
merged_ds.to_zarr('cached_output/averages_year_081_100.zarr')